In [318]:
!pip install yfinance pandas numpy scikit-learn gspread oauth2client


# now the real file

In [319]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# List of 3 NIFTY 50 stocks
stocks = ['INFY.NS','TCS.NS', 'HDFCBANK.NS']

# Define date range: last 6 months
end_date = datetime.today()
start_date = end_date - timedelta(days=180)

# Download daily data for all stocks
data = {}
for symbol in stocks:
    df = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    df.dropna(inplace=True)
    data[symbol] = df
    print(f"\n{symbol} Data Preview:")
    display(df.head())


/tmp/ipython-input-319-400241107.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed


INFY.NS Data Preview:


Price,Close,High,Low,Open,Volume
Ticker,INFY.NS,INFY.NS,INFY.NS,INFY.NS,INFY.NS
Date,,,,,
2024-12-31,1853.915161,1870.679288,1819.450138,1866.044548,3613063
2025-01-01,1856.380493,1866.685452,1847.998430,1847.998430,1838368
2025-01-02,1930.685059,1935.418507,1859.141753,1860.818117,7083645
2025-01-03,1911.850098,1925.853026,1895.332502,1925.853026,6218003
2025-01-06,1910.962524,1945.624846,1901.249216,1924.916219,7164403


/tmp/ipython-input-319-400241107.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed


TCS.NS Data Preview:


Price,Close,High,Low,Open,Volume
Ticker,TCS.NS,TCS.NS,TCS.NS,TCS.NS,TCS.NS
Date,,,,,
2024-12-31,4049.075684,4093.721663,3987.026378,4088.826744,1555429
2025-01-01,4066.528564,4087.837733,4039.829818,4048.679830,763161
2025-01-02,4129.121582,4136.290625,4051.201694,4073.994113,1720835
2025-01-03,4054.118408,4133.274820,4046.603418,4133.274820,1791953
2025-01-06,4049.273193,4103.312848,4020.992459,4059.655945,2085593


/tmp/ipython-input-319-400241107.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed


HDFCBANK.NS Data Preview:


Price,Close,High,Low,Open,Volume
Ticker,HDFCBANK.NS,HDFCBANK.NS,HDFCBANK.NS,HDFCBANK.NS,HDFCBANK.NS
Date,,,,,
2024-12-31,1753.558960,1762.411644,1746.387850,1751.036780,7058618
2025-01-01,1763.351196,1774.478781,1741.936754,1754.152345,3656903
2025-01-02,1774.231567,1783.825970,1753.707356,1757.762718,9279288
2025-01-03,1730.166260,1775.715221,1727.297912,1771.511467,12213097
2025-01-06,1691.887451,1738.079244,1687.238642,1721.165339,9408174


In [320]:
def add_indicators_manual(df):
    df = df.copy()

    # Calculate daily price change
    delta = df['Close'].diff()

    # Separate gains and losses
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)

    # Compute average gain and loss
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()

    # Avoid division by zero
    rs = avg_gain / avg_loss.replace(to_replace=0, method='ffill')

    # Calculate RSI
    df['RSI'] = 100 - (100 / (1 + rs))

    # Moving Averages
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA50'] = df['Close'].rolling(window=50).mean()

    return df


In [321]:
for symbol in data:
    data[symbol] = add_indicators_manual(data[symbol])
    print(f"\n{symbol} with Manual Indicators:")
    display(data[symbol].tail())



INFY.NS with Manual Indicators:


/tmp/ipython-input-320-3675105579.py:16: FutureWarning: The 'method' keyword in DataFrame.replace is deprecated and will be removed in a future version.
  rs = avg_gain / avg_loss.replace(to_replace=0, method='ffill')


Price,Close,High,Low,Open,Volume,RSI,MA20,MA50
Ticker,INFY.NS,INFY.NS,INFY.NS,INFY.NS,INFY.NS,,,
Date,,,,,,,,
2025-06-23,1584.000000,1601.000000,1575.400024,1601.000000,7943676,59.318182,1586.160333,1527.470435
2025-06-24,1581.599976,1609.000000,1576.599976,1601.000000,8083992,57.096175,1587.824579,1531.417959
2025-06-25,1614.900024,1619.900024,1585.099976,1590.000000,7813533,62.397702,1591.070007,1535.918079
2025-06-26,1616.000000,1624.000000,1606.599976,1608.800049,6519233,61.059312,1593.690009,1540.113792
2025-06-27,1608.000000,1629.599976,1602.400024,1619.000000,8912268,57.471258,1595.955011,1544.403923


/tmp/ipython-input-320-3675105579.py:16: FutureWarning: The 'method' keyword in DataFrame.replace is deprecated and will be removed in a future version.
  rs = avg_gain / avg_loss.replace(to_replace=0, method='ffill')



TCS.NS with Manual Indicators:


Price,Close,High,Low,Open,Volume,RSI,MA20,MA50
Ticker,TCS.NS,TCS.NS,TCS.NS,TCS.NS,TCS.NS,,,
Date,,,,,,,,
2025-06-23,3393.600098,3425.000000,3370.000000,3415.000000,3123421,52.419802,3435.931848,3419.078682
2025-06-24,3390.399902,3442.100098,3382.000000,3434.600098,3403048,51.257613,3432.058032,3422.526733
2025-06-25,3444.800049,3450.000000,3400.000000,3407.300049,1757699,58.726017,3430.953784,3427.362129
2025-06-26,3441.800049,3451.399902,3415.300049,3444.800049,2879106,56.986908,3429.679700,3431.806465
2025-06-27,3441.100098,3466.399902,3431.000000,3455.000000,1778433,52.558660,3430.090344,3435.729312



HDFCBANK.NS with Manual Indicators:


/tmp/ipython-input-320-3675105579.py:16: FutureWarning: The 'method' keyword in DataFrame.replace is deprecated and will be removed in a future version.
  rs = avg_gain / avg_loss.replace(to_replace=0, method='ffill')


Price,Close,High,Low,Open,Volume,RSI,MA20,MA50
Ticker,HDFCBANK.NS,HDFCBANK.NS,HDFCBANK.NS,HDFCBANK.NS,HDFCBANK.NS,,,
Date,,,,,,,,
2025-06-23,1927.198730,1936.595357,1920.670572,1927.198730,5869759,56.243436,1921.427228,1904.143376
2025-06-24,1939.562744,1957.465767,1934.320367,1943.618106,11312661,55.281517,1923.138403,1908.018743
2025-06-25,1958.652710,1960.433172,1942.530148,1944.607273,9202966,57.750768,1925.942554,1911.449995
2025-06-26,1999.800049,2005.042305,1958.454843,1959.839633,25042415,60.266805,1930.606250,1914.553848
2025-06-27,2014.900024,2019.800049,1993.400024,2013.000000,14106692,62.927725,1935.164410,1917.700552


In [322]:
def backtest_strategy(df):
    df = df.copy()
    df['Signal'] = 0  # 1 = Buy, -1 = Sell, 0 = Hold

    position = False
    buy_price = 0
    trades = []

    for i in range(1, len(df)):
        try:
            # Extract scalars
            rsi = df['RSI'].iloc[i].item()
            rsi_prev = df['RSI'].iloc[i-1].item()
            ma20 = df['MA20'].iloc[i].item()
            ma50 = df['MA50'].iloc[i].item()
            ma20_prev = df['MA20'].iloc[i - 1].item()
            ma50_prev = df['MA50'].iloc[i - 1].item()
            close_price = df['Close'].iloc[i].item()

            # Check for NaN
            if any(pd.isna([rsi, ma20, ma50, ma20_prev, ma50_prev])):
                continue

            # Buy condition: relaxed RSI < 40 only
            if not position and rsi < 40:
                buy_price = close_price
                position = True
                df.at[df.index[i], 'Signal'] = 1
                trades.append({'Buy Date': df.index[i], 'Buy Price': buy_price})

            # Sell condition
            elif position and (rsi > 70 or (ma20 < ma50 and ma20_prev >= ma50_prev)):
                sell_price = close_price
                position = False
                df.at[df.index[i], 'Signal'] = -1
                trades[-1].update({'Sell Date': df.index[i], 'Sell Price': sell_price})

        except Exception as e:
            print(f"Error at row {i}: {e}")
            continue

    # Compile trades
    trades_df = pd.DataFrame(trades)
    if not trades_df.empty:
        trades_df['P&L'] = trades_df['Sell Price'] - trades_df['Buy Price']
        trades_df['Win'] = trades_df['P&L'] > 0

    return df, trades_df


In [323]:
results = {}

for symbol in data:
    print(f"\n🔍 Backtesting {symbol}...")
    signals_df, trades_df = backtest_strategy(data[symbol])
    results[symbol] = {'signals': signals_df, 'trades': trades_df}

    if not trades_df.empty:
        print(f"📊 Total Trades: {len(trades_df)} | Wins: {trades_df['Win'].sum()} | Win Ratio: {trades_df['Win'].mean():.2f}")
        display(trades_df)
    else:
        print("⚠️ No trades generated.")



🔍 Backtesting INFY.NS...
📊 Total Trades: 2 | Wins: 1 | Win Ratio: 0.50


,Buy Date,Buy Price,Sell Date,Sell Price,P&L,Win
0,2025-03-11,1638.545410,2025-04-30,1479.286255,-159.259155,False
1,2025-06-04,1551.199951,2025-06-11,1631.099976,79.900024,True



🔍 Backtesting TCS.NS...
📊 Total Trades: 2 | Wins: 1 | Win Ratio: 0.50


,Buy Date,Buy Price,Sell Date,Sell Price,P&L,Win
0,2025-03-11,3543.801514,2025-04-29,3441.907227,-101.894287,False
1,2025-05-30,3432.887207,2025-06-26,3441.800049,8.912842,True



🔍 Backtesting HDFCBANK.NS...
📊 Total Trades: 1 | Wins: 1 | Win Ratio: 1.00


,Buy Date,Buy Price,Sell Date,Sell Price,P&L,Win
0,2025-03-11,1667.159424,2025-03-24,1780.413574,113.25415,True


In [324]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def run_logistic_model_on_all(results):
    print("📊 Logistic Regression Prediction Accuracy:\n")

    for symbol, data in results.items():
        try:
            df = data['signals'].copy()

            # Prepare features and target
            df[('Return', '')] = df[('Close', symbol)].pct_change().shift(-1)
            df[('Target', '')] = (df[('Return', '')] > 0).astype(int)

            # Drop NaNs
            df = df.dropna(subset=[('RSI', ''), ('MA20', ''), ('MA50', ''), ('Volume', symbol), ('Target', '')])

            X = df[[('RSI', ''), ('MA20', ''), ('MA50', ''), ('Volume', symbol)]]
            y = df[('Target', '')]

            # Split and fit model
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
            model = LogisticRegression(max_iter=1000)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)

            print(f"✅ {symbol}: Accuracy = {acc*100:.2f}%")

        except Exception as e:
            print(f"❌ {symbol}: Failed due to error: {e}")


In [325]:
run_logistic_model_on_all(results)


📊 Logistic Regression Prediction Accuracy:

✅ INFY.NS: Accuracy = 43.48%
✅ TCS.NS: Accuracy = 52.17%
✅ HDFCBANK.NS: Accuracy = 60.87%


In [326]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def run_decision_tree_model_on_all(results):
    print("🌳 Decision Tree Prediction Accuracy:\n")

    for symbol, data in results.items():
        try:
            df = data['signals'].copy()

            # Prepare return and target
            df[('Return', '')] = df[('Close', symbol)].pct_change().shift(-1)
            df[('Target', '')] = (df[('Return', '')] > 0).astype(int)

            # Drop NaNs
            df = df.dropna(subset=[('RSI', ''), ('MA20', ''), ('MA50', ''), ('Volume', symbol), ('Target', '')])

            X = df[[('RSI', ''), ('MA20', ''), ('MA50', ''), ('Volume', symbol)]]
            y = df[('Target', '')]

            # Split into training and testing
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

            # Train Decision Tree
            dt_model = DecisionTreeClassifier(max_depth=4, random_state=42)
            dt_model.fit(X_train, y_train)

            y_pred = dt_model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)

            print(f"✅ {symbol}: Accuracy = {acc*100:.2f}%")

        except Exception as e:
            print(f"❌ {symbol}: Failed due to error: {e}")


In [327]:
 run_decision_tree_model_on_all(results)

🌳 Decision Tree Prediction Accuracy:

✅ INFY.NS: Accuracy = 47.83%
✅ TCS.NS: Accuracy = 60.87%
✅ HDFCBANK.NS: Accuracy = 39.13%


In [328]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Set up credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds_path = 'algotradingfree-key.json'  # Make sure this matches your uploaded JSON file
creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
client = gspread.authorize(creds)

# 🟢 Open your sheet by correct name
sheet = client.open("AlgoTradingResults")  # ✅ MATCHES your actual sheet
print("✅ Connected to AlgoTradingResults")


✅ Connected to AlgoTradingResults


In [329]:
def log_trades_to_sheets(results, sheet, stock_symbol):
    try:
        sheet_name = f"{stock_symbol}_TRADES"
        sheet_list = [ws.title for ws in sheet.worksheets()]

        if sheet_name in sheet_list:
            ws = sheet.worksheet(sheet_name)
            ws.clear()
        else:
            ws = sheet.add_worksheet(title=sheet_name, rows="100", cols="10")

        trades_df = results[stock_symbol]['trades']
        if not trades_df.empty:
            trades_df_clean = trades_df.copy()

            # ✅ 1. Convert datetime columns to strings
            for col in trades_df_clean.columns:
                if pd.api.types.is_datetime64_any_dtype(trades_df_clean[col]):
                    trades_df_clean[col] = trades_df_clean[col].astype(str)

            # ✅ 2. Replace inf, -inf, NaN with None
            trades_df_clean = trades_df_clean.replace([float('inf'), float('-inf')], None)
            trades_df_clean = trades_df_clean.where(pd.notnull(trades_df_clean), None)

            # ✅ 3. Upload cleaned data
            ws.update([trades_df_clean.columns.tolist()] + trades_df_clean.values.tolist())
            print(f"✅ Updated: {sheet_name}")
        else:
            ws.update([["No trades found"]])
            print(f"⚠️ No trades for: {stock_symbol} — empty sheet created")

    except Exception as e:
        print(f"❌ Error updating trades for {stock_symbol}: {e}")


In [330]:
def log_summary_to_sheets(results, sheet):
    """Logs overall trade summary to Google Sheet."""
    try:
        summary_data = []

        for symbol, res in results.items():
            trades = res['trades']
            if not trades.empty:
                total_trades = len(trades)
                total_pnl = trades['P&L'].sum()
                win_ratio = trades['Win'].mean()
                summary_data.append([symbol, total_trades, total_pnl, round(win_ratio, 2)])

        sheet_list = [ws.title for ws in sheet.worksheets()]
        if 'SUMMARY' in sheet_list:
            ws = sheet.worksheet("SUMMARY")
            ws.clear()
        else:
            ws = sheet.add_worksheet(title="SUMMARY", rows="50", cols="10")

        if summary_data:
            summary_df = pd.DataFrame(summary_data, columns=['Stock', 'Total Trades', 'Total P&L', 'Win Ratio'])
            ws.update([summary_df.columns.tolist()] + summary_df.values.tolist())
            print("✅ Updated: SUMMARY")
        else:
            ws.update([["No summary available"]])
            print("⚠️ No trades to summarize — SUMMARY is empty")

    except Exception as e:
        print(f"❌ Error uploading summary: {e}")


In [334]:
# Update all trade sheets
for symbol in results.keys():
    log_trades_to_sheets(results, sheet, symbol)

# Update the summary sheet
log_summary_to_sheets(results, sheet)


✅ Updated: INFY.NS_TRADES
✅ Updated: TCS.NS_TRADES
✅ Updated: HDFCBANK.NS_TRADES
✅ Updated: SUMMARY


In [332]:
def run_algo(stock_symbols, sheet):
    import yfinance as yf
    import pandas as pd
    from datetime import datetime, timedelta

    print("🚀 Starting Algo-Trading Run...\n")
    results = {}

    # Date range: last 6 months
    end_date = datetime.today()
    start_date = end_date - timedelta(days=180)

    for symbol in stock_symbols:
        print(f"📦 Fetching: {symbol}")
        try:
            df = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
            df.dropna(inplace=True)

            # Apply manual RSI + MA
            df = add_indicators_manual(df)

            # Backtest
            signals_df, trades_df = backtest_strategy(df)

            # Store
            results[symbol] = {
                'signals': signals_df,
                'trades': trades_df
            }

            print(f"✅ Done: {symbol} — {len(trades_df)} trades")

        except Exception as e:
            print(f"❌ Error with {symbol}: {e}")

    print("\n📤 Logging results to Google Sheets...\n")

    for symbol in results:
        log_trades_to_sheets(results, sheet, symbol)

    log_summary_to_sheets(results, sheet)

    print("\n🎉 Algo Run Complete. Sheets Updated.\n")


In [338]:
stock_list = ['INFY.NS','TCS.NS', 'HDFCBANK.NS']
run_algo(stock_list, sheet)


/tmp/ipython-input-332-787747427.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed

🚀 Starting Algo-Trading Run...

📦 Fetching: INFY.NS



/tmp/ipython-input-320-3675105579.py:16: FutureWarning: The 'method' keyword in DataFrame.replace is deprecated and will be removed in a future version.
  rs = avg_gain / avg_loss.replace(to_replace=0, method='ffill')
/tmp/ipython-input-332-787747427.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed

✅ Done: INFY.NS — 2 trades
📦 Fetching: TCS.NS



/tmp/ipython-input-320-3675105579.py:16: FutureWarning: The 'method' keyword in DataFrame.replace is deprecated and will be removed in a future version.
  rs = avg_gain / avg_loss.replace(to_replace=0, method='ffill')
/tmp/ipython-input-332-787747427.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed

✅ Done: TCS.NS — 2 trades
📦 Fetching: HDFCBANK.NS



/tmp/ipython-input-320-3675105579.py:16: FutureWarning: The 'method' keyword in DataFrame.replace is deprecated and will be removed in a future version.
  rs = avg_gain / avg_loss.replace(to_replace=0, method='ffill')


✅ Done: HDFCBANK.NS — 1 trades

📤 Logging results to Google Sheets...

✅ Updated: INFY.NS_TRADES
✅ Updated: TCS.NS_TRADES
✅ Updated: HDFCBANK.NS_TRADES
✅ Updated: SUMMARY

🎉 Algo Run Complete. Sheets Updated.

